# Trabalho Prático 1 - Aprendizado de Máquina

<font size=4>Nome: Alexandre Maros<br></font>
<font size=3>PPGCC - UFMG - 2018/1</font>


Este é o _notebook_ referente ao primeiro trabalho prático da disciplina de Aprendizado de Máquina da UFMG.

O objetivo é testar uma rede de 3 camadas (1 camada de entrada, 1 camada oculta e 1 camada de saída), com diferentes configurações de hiperparâmetros, para resolver o problema de reconhecer dígitos escritos a mão (como no exemplo do dataset _mnist_)

A entrada da rede são 784 píxeis que corresponde a uma imagem de 28x28 referente ao dígito escrito. Cada píxel pode variar seu valor de 0 à 255, onde 0 é o píxel completamente preto e 255 o píxel branco. 

A saída da rede é um vetor de 10 posições, onde cada posição $i$ (0 a 9) é a probabilidade de ser o dígito $i$. Para isso utiliza-se a função de _cross-entropy_.

A função de perda a ser minimizada é a seguinte

![image.png](loss.jpg)

Este trabalho foi desenvolvido com a ajuda da bilbioteca _scikit-learn_, _pandas_, _matplotlib_ e _numpy_.

**Aviso**: A execução total desse _notebook_ é demorada!

In [ ]:
import numpy as np
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os
import time

pd.options.display.max_rows = 10

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# 1. Leitura dos Dados

Para a leitura de dados foi utilizado a biblioteca Pandas por sua facilidade em ler csv's e gerar informações dos dados caso seja necessário.

Essa etapa consiste em simplesmente ler o arquivo com as 5000 entradas e separar em dois dataframes, o dataframe X, referente as features dos dados (os valores de cada pixel de cada entrada) e outro dataframe y específico para as labels das entradas (numero correto do digito de cada entrada).

* O vetor X possui 5000 linhas e 784 colunas
* O vetor y possui 5000 linhas e 1 coluna

In [ ]:
# Encontrar Diretorio Atual
diretorio_atual = os.path.realpath('.')

# Ler o arquivo de entrada
dados = pd.read_csv(os.path.join(diretorio_atual, "data_tp1"), sep=",", header=None)

# Armazenar o label correto dos numeros
dados = dados.rename(columns = {0:'label'})
y = dados.label

# Retirar a primeira coluna referente aos labels
X = dados.drop("label", axis=1)

# Modificar o nome das colunas
num_imagens = X.shape[1]
X.columns = [ int(x) for x in range(0, num_imagens)] 

X

# 2. Visualização dos dados

O propósito desta etapa está em analisar os dados para melhor entender como os mesmos se comportam.

Inicialmente é feito a visualização das três primeiras entradas do vetor X, para verificar como é a estrutura dos dígitos e em sequência é verificado a distribuição dos dígitos dentro do dataset.

### Visualização dos três primeiro dígitos do dataset

In [ ]:
digitos = [ X.loc[0,:], X.loc[1,:], X.loc[2,:] ]
labels = [ y[0], y[1], y[2] ]

# 784 colunas correspondem a uma imagem de 28x28
plot1 = np.reshape(digitos[0].values, (28, 28))
plot2 = np.reshape(digitos[1].values, (28, 28))
plot3 = np.reshape(digitos[2].values, (28, 28))

fig=plt.figure(figsize=(8, 8))

# Plot the digits
fig.add_subplot(4, 4, 1)
plt.imshow(plot1, cmap='gray_r')
plt.title('Label: {}'.format(labels[0]))

fig.add_subplot(4, 4, 2)
plt.imshow(plot2, cmap='gray_r')
plt.title('Label: {}'.format(labels[1]))

fig.add_subplot(4, 4, 3)
plt.imshow(plot3, cmap='gray_r')
plt.title('Label: {}'.format(labels[2]))

plt.show()


### Verificando a distribuição dos dígitos

É interessante verificar a disposição (frequência) dos dígitos no dataset. Se tiver muitos digitos 1 e poucos dígitos 9 isso pode levar a uma má previsão.

In [ ]:
y.value_counts(normalize=True).sort_index()

Os dígitos parecem estar bem distribuídos, sendo que os dígitos 0 e 5 são os dois digitos que menos aparecem (9.2% dos dígitos são o dígito 0 e 0.912% dos digitos são o dígito 5). O dígito 1 é o que mais aparece, com sua frequência em 11%.

# 3. Construção dos Modelos

Esta seção descreve a construção dos modelos e uma análise sobre seus resultados. Para gerar os modelos foi utilizado princiaplemente a classe MLPClassifier da biblioteca scikit-learn do Python.

## 3.1 Feature Scaling

As redes neurais de múltiplas camandas são muito sensíveis a variância da unidade dos dados. Para diminuir esses efeitos a biblioteca scikit-learn sugere que os dados sejam "_ajustados_" de acordo com sua média e desvio padrão, de forma a diminuir os efeitos da variância e deixar os dados parecidos com uma distribuição normal [1]. Nesta etapa é exatamente isso que está sendo feito. Os numeros dos píxels estão sendo convertidos de [0,255] para [-1,+1]

[1]: Mais informações em: http://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html

In [ ]:
scaler = StandardScaler()
scaler.fit(X)
StandardScaler(copy=True, with_mean=True, with_std=True)
X = scaler.transform(X)

## 3.2 Definição das funções

Aqui está sendo definido as funções que facilitarão a análise.

A primeira função (**plot_learning_curve**) toma como principais parâmetros um modelo ainda não calculado, e as séries de treinamento.

Essa função calcula o impacto que a quantidade de dados utilizados no treinamento tem no modelo e valida essa informação utilizando **10-fold cross validation**. A quantidade de treinos utilizada vai de 10% da quantidade inicial até 100%, variando de 10% a cada etapa.

A função de "pontuação" do gráfico é a "negative log loss", logo, quanto maior o valor menos erro.

In [ ]:
def plot_learning_curve(estimator, X, y, title="Default Title", ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):
    
    fig = plt.figure(figsize=(13, 6), dpi= 80)
    plt.style.use('ggplot')

    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Exemplos de Treinamento")
    plt.ylabel("Pontuação")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y,
        cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='neg_log_loss')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Pontuação de Treinamento")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Pontuação de Cross-validation")

    plt.legend(loc="best")
    return plt


A função **plot_loss_curve** é uma simples função para desenhar a curva de perda de cada iteração do modelo.

In [ ]:
def plot_loss_curve(loss_curve_, title='Default Title'):
    fig = plt.figure(figsize=(13, 6), dpi= 80, facecolor='w', edgecolor='k')
    plt.style.use('ggplot')
    
    plt.title(title)
    
    plt.plot(loss_curve_, '-', color='blue', linewidth=3.0)
    
    plt.ylabel('Função de perda')
    plt.xlabel('Número de Iterações')
    
    return plt

## 3.3 Teste dos modelos

Aqui é testado os modelos com os diferentes hiperparâmetros. São testadas as seguintes variações:

1. Variação do Learning Rate
2. Variação do Número de Perceptrons na camada oculta
3. Variação do algoritmo de cálculo de gradiente (Gradient Descent, Stochastic Gradient Descent, Mini-Batch SGD)

Os outros hiperparâmetros são fixos, sendo eles:

1. Função de ativação: sigmoide
2. Tolerância (quando o modelo para de treinar pois atingiu convergência): 1e-4
3. Máximas iterações: 300
4. Momentum: 0.5

Cada análise de hiperparâmetro é seguido de dois gráficos, sendo o primeiro referente a curva do erro de treinamento de acordo com o número de iterações (épocas) e o segundo um gráfico de validação cruzada (usando _10-k fold_) variando a quantidade de exemplos analisados pela rede neural.

No caso do segundo gráfico, o eixo _y_ é a "pontuanção", que é o negativo da função de perda (_neg log loss_). Quanto mais próximo do 0, mais acertos o modelo tem e consequentemente melhor ele é.

### 3.3.1 Variação do learning rate

Para testar a variação do learning rate **será fixado o número de camadas na camada oculta em 50 e será usado o mini-batch SGD com 200 entradas** (esse é o _default_ do scikit-learn).

Inicialmente verifica-se como o modelo se comporta quando o learning rate está configurado para **0.5, 1 e 10**, seguido de um teste com um learning rate mais baixo (**0.01**), pois é verificado que os _learning rates_ iniciais são grandes e causam _overfitting_.

### Learning Rate = 0.5

In [ ]:
learning_rate=0.5

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nLearning Rate = {}".format(learning_rate))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nLearning Rate = {}".format(learning_rate), n_jobs=6)

Quando o _learning rate_ é configurado para ser 0.5, o modelo converge em aproximadamente 90 iterações (épocas) e atinge um erro empírico que tende a 0).

Quando realizamos a validação cruzada com diferentes quantidades de entrada, nota-se que o erro dessa validação é muito maior do que o erro de treinamento, que em todos os casos tendeu a 0. Essa situação nos mostra que possilvemente o _learning rate_ está muito elevado e o modelo está convergindo muito rápido, atingindo o estado de _overfit_.

### Learning Rate = 1

In [ ]:
learning_rate=1

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nLearning Rate = {}".format(learning_rate))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nLearning Rate = {}".format(learning_rate), n_jobs=6)

Quando aumentamos o erro em 0.5, a quantidade de iterações diminuiu para aproximadamente 58, significando que a convergência é ainda mais rápida. O erro também tende a 0 o que provavelmente vai indicar _overfitting_.

Ao analisar a validação cruzada, chega-se a mesma conclusão anterior: O erro da validação com o erro de treinamento são grandes e o erro médio piorou (embora, como pode ser visto pelo desvio padrão, ambos são bastante semelhantes).

### Learning Rate = 10

In [ ]:
learning_rate=10

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nLearning Rate = {}".format(learning_rate))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nLearning Rate = {}".format(learning_rate), n_jobs=6)

Com o _learning rate_ em 10, é possível notar que a convergência é ainda mais rápida e que também, em certos pontos da curva de _loss_, a perda aumenta ao invés de diminuir de uma iteração para outra. Isso é explicado pelo alto _learning rate_, que provavelmente fez com que o modelo passasse para além do mínimo que estava seguindo.

Na curva de validação, é possível notar que o modelo selecionado não é um bom modelo. Apesar da curva de treinamento tender a 0, a curva de validação nos mostra que a distância entre o erro empírico e o erro de validação é muito alta, e o erro no último caso ficou em torno de -1.5, o que é muito pior que os outros casos (fque ficaram em torno de -0.4)

### Learning Rate = 0.01

In [ ]:
learning_rate=0.01

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nLearning Rate = {}".format(learning_rate))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=learning_rate, max_iter=300, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nLearning Rate = {}".format(learning_rate), n_jobs=6)

Com o _learning rate_ baixo (0.01), o número de iterações para convergir aumenta significativamente (em 300 ainda não convergiu completamente) e o erro não chega a 0.

Entretanto, ao fazer a validação cruzada nota-se que a curva do erro de treinamento acompanha a curva do erro de validação (tem formas muito parecidas). Com o aumento de número de exemplos o erro vai diminuindo nos dois casos e dessa vez a diferença entre o erro empírico e o erro de validação estão mais próximos que os outros casos. O erro para esse _learning rate_ foi o menos dos 3 casos anteriores, tendendo a -0.3.

### 3.3.2 Variação do número de camadas ocultas

Nesta etapa será feito a experimentação da variação da quantidade de perceptrons na camada oculta.

Os tamanhos considerados para a camada oculta são: 25, 50 e 100.

Será fixado o **learning_rate** em 0.01 (melhor no caso anterior) e o algoritmo de gradiente em Mini-Batch SGD (tamanho da batch de 200).

Neste caso também foi aumentado o número de iterações para 1000, devido a convergência mais lenta quando o __learning_rate__ está baixo.

### Tamanho da camada oculta = 25

In [ ]:
hidden_layer_sizes=(25,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nTamanho da camada oculta = {}".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nTamanho da camada oculta = {}".format(hidden_layer_sizes), n_jobs=6)

Com o quantidade de neurônios na camada oculta em 25, o modelo demora um pouco mais de 800 iterações para convergir.

No gráfico de validação, o erro de cross-validation no último caso está em aproximadametne -0.37. O erro de treinamento não chegou a atingir 0.

### Tamanho da camada oculta = 50

In [ ]:
hidden_layer_sizes=(50,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nTamanho da camada oculta = {}".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nTamanho da camada oculta = {}".format(hidden_layer_sizes), n_jobs=6)

Com a quantidade de neurônios na camada oculta em 50, o modelo convergiu um pouco mais rápido que o de com 25 camadas, em um pouco mais de 800 iterações (épocas) e não tendeu a 0, mas um pouco acima disso.

No gráfico de validação cruzada, pode-se perceber que o erro da validação cruzada com o modelo de 50 perceptrons é um pouco melhor que o modelo com 25 perceptrons. O modelo com 50 perceptrons conseguiu uma potuação que se aproxima de -0.32, enquanto que o modelo com 25 perceptrons se aproximou de -0.37.

### Tamanho da camada oculta = 100

In [ ]:
hidden_layer_sizes=(100,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
clf_trained = clf.fit(X, y)
plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nTamanho da camada oculta = {}".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=200, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nTamanho da camada oculta = {}".format(hidden_layer_sizes), n_jobs=6)

O modelo com 100 perceptrons na camada oculta atingiu a convergência ainda mais rápido que os outros modelos (enquanto o modelo com 50 perceptrons atingiu a convergência com aproximadamente 820 iterações, o modelo com 100 percepetrons atingiu a convergência em 800 iterações, o que não é uma diferença tão grande).

No gráfico com a validação cruzada, pode-se perceber que não houve muita diferença do erro entre o modelo com 50 perceptrons e o modelo com 100, inclusive o desvio padrão do modelo com 100 perceptrons na camada oculta apresentou um desvio padrão um pouco maior nos seus erros.

Essa análise nos leva a crer que a melhor quantidade de perceptrons na camada oculta para esse problema é de 50. O modelo precisa de um pouco mais iterações para convergir mas por haver menos parâmetros para calcular, o modelo se torna um pouco mais rápido para ser computado, além de ter o menor erro dos três.

### 3.3.3 Variação do algoritmo de gradiente e número de perceptrons na camada oculta por algoritmo

Neste cenário foi considerado a variação do algoritmo de gradiente, sendo eles:

1. Gradient Descent (Calcula o erro de todos os exemplos para depois ajustar o erro)
2. Stochastic Gradient Descent (Atualiza os pesos para cada exemplo visto)
3. Mini-Batch Stochastic Gradient Descent (Atualiza os pesos para cada $m$ pesos vistos)

Além disso, aqui **também é analisado o tempo de execução de cada algoritmo**, já que os métodos possuem abordagens muito diferentes umas das outras.

Para o primeiro caso (Gradient Descent), também há a análise da variação do número de perceptrons na camada oculta (para o primeiro caso, referente ao gradient descent). Não foi feito essa análise para os outros pois nos três casos os comportamentos foram muito semelhantes.

**Para a análise do algoritmo Gradient Descent, também foi aumentado o número de iterações máximas para 5000**, pois o gradient descent tende a demorar muito mais para convergir.

**Aviso:** A execução do algoritmo Gradient Descent é demorada. 

### Gradient Descent (Tamanho da camada oculta: 25)

In [ ]:
batch_size=5000
hidden_layer_sizes=(25,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execucação (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes), n_jobs=6)

comentario...

### Gradient Descent (Tamanho da camada oculta: 50)

In [ ]:
batch_size=5000
hidden_layer_sizes=(50,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)

start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execução (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes), n_jobs=6)

comentario...

### Gradient Descent (Tamanho da camada oculta: 100)

In [ ]:
batch_size=5000
hidden_layer_sizes=(100,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)

start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execução (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=5000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nGradient Descent (Tamanho da camada oculta: {})".format(hidden_layer_sizes), n_jobs=6)

comentario...

### Stochastic Gradient Descent (Tamanho da camada oculta: 50)

In [ ]:
batch_size=1
hidden_layer_sizes=(50,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)

start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execução (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nStochastic Gradient Descent")


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=1000, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nStochastic Gradient Descent", n_jobs=6)

Com o algoritmo de Stochastic Gradient Descent, onde os pesos são atualizados a cada novo dado recebido a convergência é extremamente rápida, chegando precisar somente de 38 épocas para convergir. O erro empírico tende a 0, o que pode indicar overfitting.

No gráfico da validação cruzada é possível identificar que em todos os casos o modelo chegou a atingir um erro de 0, e o erro com a maior quantidade de exemplos de treinamento ficou entre -0.32 e -0.4, um erro bom quando analisado com os outros casos.

É interessante notar que, embora nesse exemplo o resultado tenha sido satisfatório, isso nem sempre acontece, já que pode atingir um mínimo local rápidamente.

# ** Falar do tempo em relacao ao anterior*

### Mini Batch SGD com 10 exemplos por batch

In [ ]:
batch_size=10

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=0.01, max_iter=500, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)

start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execução (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nMini-Batch Stochastic Gradient Descent ({})".format(batch_size))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=0.01, max_iter=500, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nMini-Batch Stochastic Gradient Descent ({})".format(batch_size), n_jobs=6)

Com o uso do Mini-batch SGD com 10 exemplos (os pesos são analisados após o cálculo dos erros de 10 exemplos), a convergência demora um pouco mais e também tende a 0.

Na validação cruzada, dá para se notar que em todos os casos o erro tendeu a 0 (embora não tenha chegado a 0) e a pontuação da validação cruzada ficou muito parecida quando utilizado o modelo com o algoritmo Stochastic Gradient Descent.

Quanto a análise do tempo, o modelo foi um pouco mais rápidio que o Stochastic Gradient Descent, levando 52 segundos para calcular o modelo com 5000 exemplos (Embora o SGD tenha convergido em menos iterações, nesse exemplo há menos cálculos de atualização de peso). Pelo erro ser muito parecido com o SGD, esse modelo se torna mais interessante de ser utilizado.

### Mini Batch SGD com 50 exemplos por batch

In [ ]:
batch_size=50
hidden_layer_sizes=(50,)

clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=500, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)

start = time.time()
clf_trained = clf.fit(X, y)
end = time.time()
print("Tempo de execução (segundos):", end - start)

plot_loss_curve(clf_trained.loss_curve_, title="Loss Curve\nMini-Batch Stochastic Gradient Descent ({})".format(batch_size))


clf = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=0.01, max_iter=500, momentum=0.5,
                    shuffle=True, verbose=False, tol=1e-4)
plot_learning_curve(clf, X, y, title="10-fold cross-validation\nMini-Batch Stochastic Gradient Descent ({})\n".format(batch_size), n_jobs=6)

O Mini-Batch SGD com 50 exemplos precisou de ainda mais iterações para convergiu e foi o que menos teve overfit em seus dados de treinamento em relação ao SGD e o outro Mini-Batch SGD (teve o maior valor na função de perda dos dois algoritmos).

O erro da validação cruzada também foi o melhor dos outros algoritmos, se aproximando de -0.34, com um desvio padrão também menor que os outros.

O tempo de execução também foi o menor dos algoritmos, com 38 segundos para calcular o modelo com todos os 5000 exemplos. Novamente, o número de iterações não está diretamente relacionado ao tempo, já que nesse caso há mais exemplos sendo analisados até atualizar os pesos das entradas dos perceptrons.

## 4. Conclusões

Neste trabalho foi visto...